# Filtering in the Frequency Domain
In this notebook, we will go through the basics of filtering in the frequency domain using the Fourier transform.

## Section 0. Preparing the Notebook
We start by importing the necessary libraries and then loading a sample image to work on.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# importing necessary packages
import numpy as np
from numpy import random
import cv2 as cv
from matplotlib import pyplot as plt
from solutions import *
from utils import *

In [ ]:
# loading the sample image and setting the colormap for pyplot
image = np.float64(cv.imread('data/tank.bmp', cv.IMREAD_GRAYSCALE) / 255)
plt.set_cmap('Greys_r')
_ = plt.imshow(image), plt.axis('off')

## Section 1. Fourier Transform in NumPy
We first start by exploring the ``fft`` module in NumPy. If you are not familiar with the Fourier Transform, I recommend that you first familiarize yourself with this concept. Some good sources are *Signals & Systems* by *Alan V. Oppenheim and Allan S. Willsky*, YouTube videos such as [this](https://youtu.be/1JnayXHhjlg) and [this](https://youtu.be/spUNpyF58BY), or online courses such as [Khan Academy's](https://www.khanacademy.org/science/electrical-engineering/ee-signals/ee-fourier-series/v/ee-fourier-series-intro).
The ``fft`` module uses an implementation of Fourier transform known as *fast Fourier transform*, whose time complexity is $\mathcal{O}(n \log(n))$ for 1-dimensional signals, much more efficient than the naive approach which has a time complexity of $\mathcal{O}(n^2)$.

### Section 1.1. Calculating the Fourier Transform
We start by calculating the forward Fourier transform of the given image. Note that you should use the ``fft2`` function to calculate the Fourier transform along both the image axes. Then, we show the magnitude of the Fourier transform at each frequency as an image.

**Note:** Since the Fourier transform of an image consists of complex numbers, we will only show the magnitude of the transform. Furthermore, since the difference in values is too great to be properly shown in a normal image, we have used the logarithm of the magnitude.

In [ ]:
# Calculate the fourier transform
image_f_ref = np.fft.fft2(image)

# Showing the magnitude map
eps = 1e-15
_ = plt.figure(figsize=(8, 8))
_ = plt.imshow(np.log(np.abs(image_f_ref) + eps)), plt.axis('off'), plt.title('Magnitude of the Fourier Transform')

### Section 1.2. Centering the Transform
As you can see above, the highest frequencies are located in the corners of the image. This is because for a sequence of length $N$, NumPy will order the transform in a way that the frequencies are ordered from $0$ to $\large \frac{2 \pi (N-1)}{N}$. However, it would be more convenient for us if all the low frequencies (that is the ones whose absolute value is closer to zero) were put in the center of the image. We can center the low frequencies using the ``np.fft.fftshift`` function.

In [ ]:
# Shifting the Fourier transform
image_f_shift_ref = np.fft.fftshift(image_f_ref)

# Showing the magnitude map
eps = 1e-15
_ = plt.figure(figsize=(8, 8))
_ = plt.imshow(np.log(np.abs(image_f_shift_ref) + eps)), plt.axis('off'), plt.title('Magnitude of the Fourier Transform')

As you can see, the low frequencies, which are the strongest in this image, are now placed at the center.

### Section 1.3. Inverse Fourier Transform
Let's say you've manipulated the frequencies of the image in the way that you want, and now you want to convert the Fourier transform back into an image. You can do so by first reversing the shift by ``np.fft.ifftshift``, then applying the inverse fourier transform by ``np.fft.ifft2``, and finally, getting rid of the complex part of the image (which should be practically equal to zero) by either using the ``np.real`` function, the ``np.abs`` function, or just casting it into ``np.float64``.

In [ ]:
# Reversing the Fourier transform
image_r = np.fft.ifft2(np.fft.ifftshift(image_f_shift_ref))
image_r = np.real(image_r)

# Showing the reconstructed image
_ = plt.figure(figsize=(8, 8))
_ = plt.imshow(image_r, vmin=0, vmax=1), plt.axis('off'), plt.title('Magnitude of the Fourier Transform')

## Section 2. Filtering in the Frequency Domain
Filtering in the frequency domain is done by multiplying two fourier transforms with each other, rather than using correlation. Let's have a look at how this is done.

### Section 2.1. Padding and Transformation
The first thing that you should do before applying the filter, is padding the image, either by zeros, copied pixels, or mirrored versions of itself. The reason for this, is that the Fourier transform inherently treats the image as a periodic function, i.e. a tiled version of itself. Therefore, if you don't pad the image, you might sometimes see data from the opposite sides *leaking* into the image, and causing unwanted artifacting. Here we will create a padded version with twice the size of the original image. And only then, will we apply the fourier transform.

In [ ]:
# Padding the image
def padAndTransform(image : np.ndarray) -> np.ndarray:
    """
    Parameters:
    - image : np.ndarray
        A grayscale image whose Fourier transform will be calculated.
    Returns:
    - output : np.ndarray
        The shifted fourier transform of the padded image, with twice the size.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()

# Calculating the Fourier transform of the padded image
image_pad_f = padAndTransform(image)
image_pad_f_ref = padAndTransformRef(image)

# Showing the results
_ = plt.figure(figsize=(14, 7))
_ = plt.subplot(1, 2, 1), plt.imshow(np.log(np.abs(image_pad_f))), plt.axis('off'), plt.title('Your Fourier Transform')
_ = plt.subplot(1, 2, 2), plt.imshow(np.log(np.abs(image_pad_f_ref))), plt.axis('off'), plt.title('Reference Fourier Transform')

### Section 2.2. Applying the Filter
Now that we have the Fourier transform, we can multiply it by a filter. Here we use a simple low-pass filter, which you will implement yourself later on in this notebook. Then, we apply the inverse transform and view the image without the padding.

In [ ]:
# Generating the low-pass filter
filter_low_pass = lowPassFilter(image_pad_f_ref.shape)

# Multiplying by the filter
image_pad_f_filtered = image_pad_f_ref * filter_low_pass

# Recreating the image
h, w = image.shape
image_filtered_ref = np.real(np.fft.ifft2(np.fft.ifftshift(image_pad_f_filtered)))[:h,:w]

# Showing the filtered image
_ = plt.figure(figsize=(14, 7))
_ = plt.subplot(1, 2, 1), plt.imshow(image_filtered_ref, vmin=0, vmax=1), plt.axis('off'), plt.title('Filtered Image')
_ = plt.subplot(1, 2, 2), plt.imshow(image, vmin=0, vmax=1), plt.axis('off'), plt.title('Original Image')

As you can see, the result is indeed a version of the image with diminished high frequencies (a.k.a fine details). You can see the dark edges of the image, which were caused by our zero-padding.

Below, you can see the result of filtering with a variety of paddings, and filtering with no padding. Notice how the details on the edges of the image change based on the padding method.

In [ ]:
image_clown = np.float64(cv.imread('data/clown.bmp', cv.IMREAD_GRAYSCALE) / 255)
paddingDemo(image_clown)

Considering these results, implement a padding function of your choice. You can choose any padding of your choice, but the solutions use mirror padding. Therefore, if you get slightly different results, it might be because you chose a different method of padding.

In [ ]:
def pad(image : np.ndarray) -> np.ndarray:
    """
    Parameters:
    - image : np.ndarray
        A grayscale image whose Fourier transform will be calculated.
    Returns:
    - output : np.ndarray
        The padded image, with twice the size.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()

# Padding an image
image_pad = pad(image)
image_pad_ref = padRef(image)

# Showing the results
_ = plt.figure(figsize=(14, 7))
_ = plt.subplot(1, 2, 1), plt.imshow(image_pad, vmin=0, vmax=1), plt.axis('off'), plt.title('Your Padded Image')
_ = plt.subplot(1, 2, 2), plt.imshow(image_pad_ref, vmin=0, vmax=1), plt.axis('off'), plt.title('Reference Padded Image')

## Section 3. The Relation Between Spatial and Frequency Filtering
If you're familiar with the concept of convolution and how it translates into the frequency domain, you'd know that these two equations are equivalent:

$
(h \star g)(t)
$

$
\large \mathcal{F}^-1[\normalsize H(\omega) \cdot G(\omega) \large]
$

And since convolution is very similar to the way we apply kernels to images (correlation with a kernel can be seen as convolution with the flipped kernel), filtering in the frequency domain is practically equivalent to filtering in the spatial domain. Meaning that if you can achieve a result by using a filter in the frequency domain, there is a way of achieving that in the spatial domain too, albeit with a different level of complexity.

### Section 3.1. Spatial Filters in the Frequency Domain
Let's begin by observing how some of the more familiar spatial kernels look like, when we take them into the frequency domain. Note that here, we have padded the kernels with zeros to be as large as the original image. Doing so does not change the effects of the kernel, since its value can be considered zero outside of the kernel boundaries.

In [ ]:
spatialKernelDemo(image.shape)

As you can see, the filters which we know as *low-pass*, diminish the high frequencies, while keeping the lower ones, and the high-pass filters do the opposite. A distinction that can be made between the Laplacian filter and the Prewitt filters, is that while the Lpalacian acts as a high-pass filter for all frequencies, the horizontal prewitt filter, for example, acts as a high-pass filter for the horizontal frequencies (those that have a high frequency in the horizontal direction, but a low frequency in the vertical direction), but for vertical frequencies, it acts as a low-pass filter. This observation is consistent with the method that we create a Prewitt filter (mean filtering in one direction and differentiating in the other).

## Section 4. Constructing Low-Pass Filters
Now that you have been familiarized with the spectra of common spatial filters, we can begin to construct some low-pass filters in the frequency domain. We can later use these to create a variety of new filters.

### Section 4.1. Ideal Low-Pass Filtering
The most straightforward approach to low-pass filtering, is simply removing all the frequencies above a certain threshold, which is known as an ideal low-pass filter. Implement such a filter, and see its results.

In [ ]:
def idealLowPass(image : np.ndarray, threshold : float) -> np.ndarray:
    """
    Parameters:
    - image : np.ndarray
        An image on which the filtering will be applied. It should be a np.ndarray
        with dtype=float64.
    - threshold : float
        The highest frequency which should be allowed to pass. Should be a float in
        [0 inf).
    Returns:
    - output : np.ndarray
        The filtered image, with dtype=np.float64.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()

# Applying the filter
threshold = 0.2 * np.pi
image_filtered = idealLowPass(image, threshold)
image_filtered_ref = idealLowPassRef(image, threshold)

# Showing the results
_ = plt.figure(figsize=(18, 6))
_ = plt.subplot(1, 3, 1), plt.imshow(image_filtered, vmin=0, vmax=1), plt.axis('off'), plt.title('Your Filtered Image')
_ = plt.subplot(1, 3, 2), plt.imshow(image_filtered_ref, vmin=0, vmax=1), plt.axis('off'), plt.title('Reference Filtered Image')
_ = plt.subplot(1, 3, 3), plt.imshow(image, vmin=0, vmax=1), plt.axis('off'), plt.title('Original Image')

As you decrease the passing threshold, you can see that a sort of rippling effect appears in the image, which is not desirable. Therefore, ideal low-pass filters (or any kind of filter that sharply blocks out certain frequencies) are generally avoided in image processing.

### Gaussian Low-Pass Filter
One alternative to the naive approach to low-pass filtering, is by using a Gaussian function. In this method, the extent to which a frequency is passes is proportional to the Gaussian function of that frequency. The equation for the filter is written below.

$
H(u,v) = \exp(- \large \frac{(u^2 + v^2)}{2\sigma^2})
$

Note that we have omitted the coefficient behind the exponential function. This is done so that $H(0,0)=1$, therefore keeping the average intensity of the image constant. 

A more *correct* way of implementing this filter in a discrete domain (e.g. our pixelated image), would contain an integration of the Gaussian function instead of the function itself. However, for most practical purposes, and as long as the $\sigma$ is not too small, or the image resolution too low, this would not be much of an issue. With these points in consideration, try implementing a Gaussian Low Pass filter.

**Note:** While this filter and the spatial Gaussian filter share the same name, and have similar spectra and kernel representations, they are often *not* equivalent. This is due to issues in the implementation, such as the limited size of the Gaussian kernel, and the simplification that we mentioned earlier. In an [ideal situation](https://mathworld.wolfram.com/FourierTransformGaussian.html), however, both the kernel and the spectrum are Gaussian functions, with inverse $\sigma$s.

In [ ]:
def gaussianLowPass(image : np.ndarray, sigma : float) -> np.ndarray:
    """
    Parameters:
    - image : np.ndarray
        An image on which the filtering will be applied. It should be a np.ndarray
        with dtype=float64.
    - sigma : float
        The sigma value for the Gaussian function. Should be a positive number.
    Returns:
    - output : np.ndarray
        The filtered image, with dtype=np.float64.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()

# Applying the filter
sigma = 0.3 * np.pi
image_gaussian_filtered = gaussianLowPass(image, sigma)
image_gaussian_filtered_ref = gaussianLowPassRef(image, sigma)

# Showing the results
_ = plt.figure(figsize=(18, 6))
_ = plt.subplot(1, 3, 1), plt.imshow(image_gaussian_filtered, vmin=0, vmax=1), plt.axis('off'), plt.title('Your Gaussian Filtered Image')
_ = plt.subplot(1, 3, 2), plt.imshow(image_gaussian_filtered_ref, vmin=0, vmax=1), plt.axis('off'), plt.title('Reference Gaussian Filtered Image')
_ = plt.subplot(1, 3, 3), plt.imshow(image, vmin=0, vmax=1), plt.axis('off'), plt.title('Original Image')

### Butterworth Low-Pass Filter
Another widely-used low-pass filter is the butterworth low-pass filter. The equation for this filter is written below.

$
H(u,v) = \Large \frac{1}{1+((u^2 + v^2) / D_0^2)^{n}}
$

Here, $D_0$ is the threshold for letting a frequency get through, and $n$ is the order of the filter. The higher order a Butterworth filter is, the sharper it will remove the unwanted frequencies. Implement and view the output of this filter with different parameters.

In [ ]:
def butterworthLowPass(image : np.ndarray, d : float, order : float) -> np.ndarray:
    """
    Parameters:
    - image : np.ndarray
        An image on which the filtering will be applied. It should be a np.ndarray
        with dtype=float64.
    - d : float
        The d value for the Butterworth filter. Should be a positive number.
    - order : float
        The order of the Butterworth filter. Should be a positive number.
    Returns:
    - output : np.ndarray
        The filtered image, with dtype=np.float64.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()

# Applying the filter
d = 0.2 * np.pi
order = 1
image_butterworth_filtered = butterworthLowPass(image, d, order)
image_butterworth_filtered_ref = butterworthLowPassRef(image, d, order)

# Showing the results
_ = plt.figure(figsize=(18, 6))
_ = plt.subplot(1, 3, 1), plt.imshow(image_butterworth_filtered, vmin=0, vmax=1), plt.axis('off'), plt.title('Your Butterworth Filtered Image')
_ = plt.subplot(1, 3, 2), plt.imshow(image_butterworth_filtered_ref, vmin=0, vmax=1), plt.axis('off'), plt.title('Reference Butterworth Filtered Image')
_ = plt.subplot(1, 3, 3), plt.imshow(image, vmin=0, vmax=1), plt.axis('off'), plt.title('Original Image')

## Section 5. Creating More Complex Filters

### Section 5.1. High-Pass Filters
One simple trick that you can do with a low-pass filter, is subtracting it from $1$ to get a high-pass filter. Doing so is equivalent to filtering with a low-pass kernel, and then subtracting the output from the image, which leaves out only the high frequency features. Apply this technique to your Gaussian and Butterworth filters, and view the results of applying a high-pass filter on the image.

In [ ]:
def gaussianHighPass(image : np.ndarray, sigma : float) -> np.ndarray:
    """
    Parameters:
    - image : np.ndarray
        An image on which the filtering will be applied. It should be a np.ndarray
        with dtype=float64.
    - sigma : float
        The sigma value for the Gaussian function. Should be a positive number.
    Returns:
    - output : np.ndarray
        The filtered image, with dtype=np.float64.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()

def butterworthHighPass(image : np.ndarray, d : float, order : float) -> np.ndarray:
    """
    Parameters:
    - image : np.ndarray
        An image on which the filtering will be applied. It should be a np.ndarray
        with dtype=float64.
    - d : float
        The d value for the Butterworth filter. Should be a positive number.
    - order : float
        The order of the Butterworth filter. Should be a positive number.
    Returns:
    - output : np.ndarray
        The filtered image, with dtype=np.float64.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()

# Applying the filters
sigma = 0.2 * np.pi
d = 0.2 * np.pi
order = 1
image_highpass_butterworth_filtered = butterworthHighPass(image, d, order)
image_highpass_butterworth_filtered_ref = butterworthHighPassRef(image, d, order)
image_highpass_gaussian_filtered = gaussianHighPass(image, sigma)
image_highpass_gaussian_filtered_ref = gaussianHighPassRef(image, sigma)

# Showing the results
_ = plt.figure(figsize=(15, 15))
_ = plt.subplot(2, 2, 1), plt.imshow(np.abs(image_highpass_butterworth_filtered)), plt.axis('off'), plt.title('Your High-Pass Butterworth Filtered Image')
_ = plt.subplot(2, 2, 2), plt.imshow(np.abs(image_highpass_butterworth_filtered_ref)), plt.axis('off'), plt.title('Reference High-Pass Butterworth Filtered Image')
_ = plt.subplot(2, 2, 3), plt.imshow(np.abs(image_highpass_gaussian_filtered)), plt.axis('off'), plt.title('Your High-Pass Gaussian Filtered Image')
_ = plt.subplot(2, 2, 4), plt.imshow(np.abs(image_highpass_gaussian_filtered_ref)), plt.axis('off'), plt.title('Reference High-Pass Gaussian Filtered Image')

### Section 5.2. Band-Pass and Band-Reject Filters
In some cases, we need to only pass or block a certain frequency in the image. In these cases, band-reject or band-pass filters are used. We will start with creating a band reject filter, and then use it to create a band-pass filter.

A naive way of implementing a band-reject filter, would be by using a hlow-pass and high-pass filter together. Say that we want to reject the frequencies in $(f_0-W/2 \;\;\; f_0+W/2)$, we can define our band-reject filter as the following.

$
BRF_{f_0,W} = LPF_{f_0-W/2} + HPF_{f_0+W/2}
$

There are several issues with such a filter. While it would work perfectly with the ideal low-pass and high-pass filter, we don't want to use such filters for reasons mentioned above. The graph of a Gaussian band-reject filter's cross-section is shown below.

In [ ]:
f = 0.5 * np.pi
W = 0.2 * np.pi
drawGaussianCrossSection(f, W, resolution=40)

As you can see, this makes for a very inadequate band-reject filter. Firstly, the minimum is not positioned on $f$, secondly, the rejected frequencies are not sufficiently weakened, and lastly, the cut-off frequencies of $f\pm W$ are very asymmetric. We can improve this filter by using another equation for the exponent of the Gaussian function.

$
g_{f,W}(z) = \Large(\frac{z^2 - f^2}{z W})\large^2
$

As you can see, such a function would be equal to 0 for $z^2 = f^2$, would symmetrically increase with a $z$ higher or lower than $f$, and will go towards infinity for large or small $z$s. We can use this function to build a Gaussian or a Butterworth band-reject filter.

$
GBRF_{f,W}(u,v) = 1 - \exp (-g_{f,W}(\sqrt{u^2+v^2}))
$


$
BBRF_{f,W,n}(u,v) = 1 - \Large \frac{1}{1 + g_{f,W}^n(\sqrt{u^2+v^2})}
$

Implement these filters, and see their results.

In [ ]:
def gaussianBandReject(image : np.ndarray, f : float, W : float) -> np.ndarray:
    """
    Parameters:
    - image : np.ndarray
        An image on which the filtering will be applied. It should be a np.ndarray
        with dtype=float64.
    - f : float
        The reject frequency for the filter. Should be a positive number.
    - W : float
        The width of the rejected band. Should be a positive number and lower than
        2f.
    Returns:
    - output : np.ndarray
        The filtered image, with dtype=np.float64.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()

def butterworthBandReject(image : np.ndarray, f : float, W : float, order : float) -> np.ndarray:
    """
    Parameters:
    - image : np.ndarray
        An image on which the filtering will be applied. It should be a np.ndarray
        with dtype=float64.
    - f : float
        The reject frequency for the filter. Should be a positive number.
    - W : float
        The width of the rejected band. Should be a positive number and lower than
        2f.
    - order : float
        The order of the Butterworth filter. should be a positive number.
    Returns:
    - output : np.ndarray
        The filtered image, with dtype=np.float64.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()

# Applying the filters
f = 0.5 * np.pi
W = 0.5 * np.pi
order = 2
image_bandreject_gaussian_filtered = gaussianBandReject(image, f, W)
image_bandreject_gaussian_filtered_ref = gaussianBandRejectRef(image, f, W)
image_bandreject_butterworth_filtered = butterworthBandReject(image, f, W, order)
image_bandreject_butterworth_filtered_ref = butterworthBandRejectRef(image, f, W, order)

# Showing the results
_ = plt.figure(figsize=(15, 15))
_ = plt.subplot(2, 2, 1), plt.imshow(np.abs(image_bandreject_gaussian_filtered)), plt.axis('off'), plt.title('Your Gaussian Band-Reject Filtered Image')
_ = plt.subplot(2, 2, 2), plt.imshow(np.abs(image_bandreject_gaussian_filtered_ref)), plt.axis('off'), plt.title('Reference Gaussian Band-Reject Filtered Image')
_ = plt.subplot(2, 2, 3), plt.imshow(np.abs(image_bandreject_butterworth_filtered)), plt.axis('off'), plt.title('Your Butterworth Band-Reject Filtered Image')
_ = plt.subplot(2, 2, 4), plt.imshow(np.abs(image_bandreject_butterworth_filtered_ref)), plt.axis('off'), plt.title('Reference Butterworth Band-Reject Filtered Image')

Similar to how you created high-pass filters from low-pass ones, you can now use your filters to create band-pass filters, which only allow the frequencies in a certain band to pass.

In [ ]:
def gaussianBandPass(image : np.ndarray, f : float, W : float) -> np.ndarray:
    """
    Parameters:
    - image : np.ndarray
        An image on which the filtering will be applied. It should be a np.ndarray
        with dtype=float64.
    - f : float
        The reject frequency for the filter. Should be a positive number.
    - W : float
        The width of the rejected band. Should be a positive number and lower than
        2f.
    Returns:
    - output : np.ndarray
        The filtered image, with dtype=np.float64.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()

def butterworthBandPass(image : np.ndarray, f : float, W : float, order : float) -> np.ndarray:
    """
    Parameters:
    - image : np.ndarray
        An image on which the filtering will be applied. It should be a np.ndarray
        with dtype=float64.
    - f : float
        The reject frequency for the filter. Should be a positive number.
    - W : float
        The width of the rejected band. Should be a positive number and lower than
        2f.
    - order : float
        The order of the Butterworth filter. should be a positive number.
    Returns:
    - output : np.ndarray
        The filtered image, with dtype=np.float64.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()

# Applying the filters
f = 0.75 * np.pi
W = 1 * np.pi
order = 2
image_bandpass_gaussian_filtered = gaussianBandPass(image, f, W)
image_bandpass_gaussian_filtered_ref = gaussianBandPassRef(image, f, W)
image_bandpass_butterworth_filtered = butterworthBandPass(image, f, W, order)
image_bandpass_butterworth_filtered_ref = butterworthBandPassRef(image, f, W, order)

# Showing the results
_ = plt.figure(figsize=(15, 15))
_ = plt.subplot(2, 2, 1), plt.imshow(np.abs(image_bandpass_gaussian_filtered)), plt.axis('off'), plt.title('Your Gaussian Band-Pass Filtered Image')
_ = plt.subplot(2, 2, 2), plt.imshow(np.abs(image_bandpass_gaussian_filtered_ref)), plt.axis('off'), plt.title('Reference Gaussian Band-Pass Filtered Image')
_ = plt.subplot(2, 2, 3), plt.imshow(np.abs(image_bandpass_butterworth_filtered)), plt.axis('off'), plt.title('Your Butterworth Band-Pass Filtered Image')
_ = plt.subplot(2, 2, 4), plt.imshow(np.abs(image_bandpass_butterworth_filtered_ref)), plt.axis('off'), plt.title('Reference Butterworth Band-Pass Filtered Image')

**Note:** There are some methods of filtering in the frequency domain which are mainly aimed at removing noise patterns. You can read about these methods and try them in the "Noise Removal" notebook.

# Scratchpad
You can use this section to try out different codes, without making a mess of the notebook. :)